# RecMetrics Example

In [ ]:
!pip install surprise
!pip install funcsigs
!pip install ml_metrics

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
import json
import recmetrics
from tqdm import tqdm
from ast import literal_eval
import ml_metrics as metrics
import sys

In [ ]:
sys.path.append("/content/drive/MyDrive/Evaluation/")

In [ ]:
triplets = pd.read_csv("/content/drive/MyDrive/Models/filtered_triplets.csv")

Plot the distribution of user-song-play triplets

In [ ]:
fig = plt.figure(figsize=(15, 7))
recmetrics.long_tail_plot(df=triplets, 
             item_id_column="song", 
             interaction_type="play count", 
             percentage=0.5,
             x_labels=False)

In [ ]:
cf_pred = pd.read_csv("/content/drive/MyDrive/Collaborative Filtering/CF_predictions.csv")
cf_pred.rename(columns = {'play_count':'actual','predicted':'cf_predictions'}, inplace = True)
cf_pred.head()

 Evaluate model with MSE and RMSE

In [ ]:
print("MSE: ", recmetrics.mse(cf_pred.actual, cf_pred.cf_predictions))
print("RMSE: ", recmetrics.rmse(cf_pred.actual, cf_pred.cf_predictions))

Create a dataframe with list of recmmendation for every user for every model

In [ ]:
val = 0 #val = 1 uses the test set of songs for all user to calculate metrics
actual_data_triplets = pd.read_csv('/content/drive/MyDrive/Models/filtered_triplets.csv')
actual_data = actual_data_triplets.groupby('user', as_index=False)['song'].agg({'actual': (lambda x: list(set(x)))})
actual_data = actual_data.set_index('user')
with open("/content/drive/MyDrive/NGCF_Float64/Data/MSD/test_actual.json") as fp:
    actual_test = json.load(fp)
f = open('cf_recommendations.json','r')
cf_recommendations = json.load(f)
f.close()
cf_dic = {}
count = 0
#list of all the songs the user has listened to for all users in a dataframe
for i in tqdm(sorted(cf_recommendations.keys()), ncols=100):
    row_dic = {}
    row_dic['userId'] = i

    if val == 1:
        if i not in actual_test:
            row_dic['actual'] = []
        else:
            row_dic['actual'] = actual_test[i]
    else:
        row_dic['actual'] = actual_data['actual'][i]

    row_dic['cf_predictions'] = cf_recommendations[i]

    cf_dic[count] = row_dic
    count += 1
cf_df = pd.DataFrame.from_dict(cf_dic, orient='index')
# cf_df = cf_df.set_index('userId')
print(cf_df.shape)

In [ ]:
f2 = open('/content/drive/MyDrive/Recommendations/content_top10.json','r')

content_recommendations = json.load(f2)
f2.close()
cf_df = cf_df[cf_df['userId'].isin(set(content_recommendations.keys()))].iloc[:,:].reset_index(drop=True)
cf_df['content_predictions'] = [content_recommendations[i] for i in sorted(content_recommendations.keys())]
print(cf_df.shape)

In [ ]:
path = "/content/drive/MyDrive/Recommendations/"

In [ ]:
ngcf_top10_df = pd.read_csv(path+"ngcf_xavier30_full_top10.csv",converters={'ngcf_predictions': literal_eval})
filtered_users = set(ngcf_top10_df['user'])
cf_df = cf_df[cf_df['userId'].isin(filtered_users)].iloc[:,:].reset_index(drop=True)
ngcf_top10_df = ngcf_top10_df.sort_values(by=['user']).reset_index(drop=True)
cf_df['ngcf_predictions'] = ngcf_top10_df['ngcf_predictions']

In [ ]:
ngcf_embedding_top10_df = pd.read_csv(path+"ngcf_embedding30_full_top10_user.csv",converters={'ngcf_embedding_predictions': literal_eval})
filtered_users = set(ngcf_embedding_top10_df['user'])
cf_df = cf_df[cf_df['userId'].isin(filtered_users)].iloc[:,:].reset_index(drop=True)
ngcf_embedding_top10_df = ngcf_embedding_top10_df.sort_values(by=['user']).reset_index(drop=True)
cf_df['ngcf_embedding_predictions'] = ngcf_embedding_top10_df['ngcf_embedding_predictions']
# cf_df

In [ ]:
test = cf_df.copy(deep=True)

Popularity Recommender

In [ ]:
orig_play_df = pd.read_csv('/content/drive/MyDrive/Models/filtered_triplets.csv')
orig_play_df
pop_df = orig_play_df.groupby(['song'],as_index=False).sum()
popularity_recs = list(pop_df.sort_values(by = ["play_count"],ascending=False).reset_index(drop=True)['song'][:10])

pop_recs = []
for user in test.index:
    pop_predictions = popularity_recs
    pop_recs.append(pop_predictions)
        
test['pop_predictions'] = pop_recs

Random Recommender

In [ ]:
#make recommendations for all members in the test data
test = test.set_index('userId') 
ran_recs = []
c = 1
for user in tqdm(test.index, ncols=100):
    random_predictions = pop_df.song.sample(10, random_state=c).values.tolist()
    ran_recs.append(random_predictions)
    c += 1
test['random_predictions'] = ran_recs
# test.head()

Recall

In [ ]:
actual = test.actual.values.tolist()
cf_predictions = test.cf_predictions.values.tolist()
pop_predictions = test.pop_predictions.values.tolist()
random_predictions = test.random_predictions.values.tolist()
content_predictions = test.content_predictions.values.tolist()
ngcf_predictions = test.ngcf_predictions.values.tolist()
ngcf_embedding_predictions = test.ngcf_embedding_predictions.values.tolist()

In [ ]:
cf_mark = []
cf_mapk = []
for K in np.arange(1, 11):
    cf_mark.extend([recmetrics.mark(actual, cf_predictions, k=K)])
    cf_mapk.extend([metrics.mapk(actual, cf_predictions, k=K)])
pop_mark = []
pop_mapk = []
for K in np.arange(1, 11):
    pop_mark.extend([recmetrics.mark(actual, pop_predictions, k=K)])
    pop_mapk.extend([metrics.mapk(actual, pop_predictions, k=K)])
random_mark = []
random_mapk = []
for K in np.arange(1, 11):
    random_mark.extend([recmetrics.mark(actual, random_predictions, k=K)])
    random_mapk.extend([metrics.mapk(actual, random_predictions, k=K)])
content_mark = []
content_mapk = []
for K in np.arange(1, 11):
    content_mark.extend([recmetrics.mark(actual, content_predictions, k=K)])
    content_mapk.extend([metrics.mapk(actual, content_predictions, k=K)])
ngcf_mark = []
ngcf_mapk = []
for K in np.arange(1, 11):
    ngcf_mark.extend([recmetrics.mark(actual, ngcf_predictions, k=K)])
    ngcf_mapk.extend([metrics.mapk(actual, ngcf_predictions, k=K)])
ngcf_embedding_mark = []
ngcf_embedding_mapk = []
for K in np.arange(1, 11):
    ngcf_embedding_mark.extend([recmetrics.mark(actual, ngcf_embedding_predictions, k=K)])
    ngcf_embedding_mapk.extend([metrics.mapk(actual, ngcf_embedding_predictions, k=K)])

Mark Plot

In [ ]:
index = range(1,10+1)
if val == 0:
    mark_scores = [random_mark, pop_mark, cf_mark, content_mark, ngcf_mark, ngcf_embedding_mark]
    names = ['Random Recommender', 'Popularity Recommender', 'Collaborative Filtering', 'Content Based Recommender', 'NGCF (Random Embeddings)', 'NGCF (Custom Embeddings)']
else:
    mark_scores = [ngcf_mark, ngcf_embedding_mark]
    names = ['NGCF (Random Embeddings)', 'NGCF (Custom Embeddings)']

fig = plt.figure(figsize=(15, 7))
recmetrics.mark_plot(mark_scores, model_names=names, k_range=index)

In [ ]:
index = range(1,10+1)
if val == 0:
    mapk_scores = [random_mapk, pop_mapk, cf_mapk, content_mapk, ngcf_mapk, ngcf_embedding_mapk]
    names = ['Random Recommender', 'Popularity Recommender', 'Collaborative Filtering', 'Content Based Recommender', 'NGCF (Random Embeddings)', 'NGCF (Custom Embeddings)']
else:
    mapk_scores = [ngcf_mapk, ngcf_embedding_mapk]
    names = ['NGCF (Random Embeddings)', 'NGCF (Custom Embeddings)']


fig = plt.figure(figsize=(15, 7))
recmetrics.mapk_plot(mapk_scores, model_names=names, k_range=index)

Prediction Coverage

In [ ]:
catalog = actual_data_triplets.song.unique().tolist()
random_coverage = recmetrics.prediction_coverage(ran_recs, catalog)
pop_coverage = recmetrics.prediction_coverage(pop_recs, catalog)
cf_coverage = recmetrics.prediction_coverage(cf_predictions, catalog)
content_coverage = recmetrics.prediction_coverage(content_predictions, catalog)
ngcf_coverage = recmetrics.prediction_coverage(ngcf_predictions,catalog)
ngcf_embedding_coverage = recmetrics.prediction_coverage(ngcf_embedding_predictions,catalog)

Coverage Plot

In [ ]:
# plot of prediction coverage
coverage_scores = [random_coverage, pop_coverage, cf_coverage, content_coverage, ngcf_coverage, ngcf_embedding_coverage]
model_names = ['Random Recommender', 'Popularity Recommender', 'Collaborative Filtering', 'Content Based Recommender', 'NGCF (Random Embeddings)', 'NGCF (Custom Embeddings)']

fig = plt.figure(figsize=(17, 7))
recmetrics.coverage_plot(coverage_scores, model_names)

Novelty

In [ ]:
users = list(set(actual_data_triplets['user']))

In [ ]:
nov = actual_data_triplets.song.value_counts()
pop = dict(nov)

In [ ]:
random_novelty,random_mselfinfo_list = recmetrics.novelty(ran_recs, pop, len(users), 10)
pop_novelty,pop_mselfinfo_list = recmetrics.novelty(pop_recs, pop, len(users), 10)
cf_novelty,cf_mselfinfo_list = recmetrics.novelty(cf_predictions, pop, len(users), 10)
content_novelty,content_mselfinfo_list = recmetrics.novelty(content_predictions, pop, len(users), 10)
ngcf_novelty,ngcf_mselfinfo_list = recmetrics.novelty(ngcf_predictions, pop, len(users), 10)
ngcf_embedding_novelty,ngcf_embedding_mselfinfo_list = recmetrics.novelty(ngcf_embedding_predictions, pop, len(users), 10)

In [ ]:
print("Novelty")
print("Random :",random_novelty)
print("Popularity :", pop_novelty)
print("Collaborative Filtering :", cf_novelty)
print("Content Based Recommendation :", content_novelty)
print("NGCF :", ngcf_novelty)
print("NGCF Embedding :", ngcf_embedding_novelty)

Mean Reciprocal Rank

In [ ]:
lis_actual_set = [set(i) for i in list(test["actual"])]
def MRR(predicted, actual = lis_actual_set):
    rr_sum = 0
    for user in range(len(predicted)):
        rr = 0
        for i in range(len(predicted[user])):
            if predicted[user][i] in actual[user]:
                rr = 1/(i+1)
                break
        rr_sum += rr
    return rr_sum/len(predicted)

In [ ]:
random_mrr = MRR(test["random_predictions"])
pop_mrr = MRR(test["pop_predictions"])
cf_mrr = MRR(test["cf_predictions"])
content_mrr = MRR(test["content_predictions"])
ngcf_mrr = MRR(test["ngcf_predictions"])
ngcf_embedding_mrr = MRR(cf_df["ngcf_embedding_predictions"])
print("Mean Reciprocal Rank")
print("Random :",random_mrr)
print("Popularity :", pop_mrr)
print("Collaborative Filtering :", cf_mrr)
print("Content Based Recommendation :", content_mrr)
print("NGCF :", ngcf_mrr)
print("NGCF Embedding :", ngcf_embedding_mrr)

Personalization

In [ ]:
random_personalization = recmetrics.personalization(predicted=random_predictions)
pop_personalization = recmetrics.personalization(predicted=pop_predictions)
cf_personalization = recmetrics.personalization(predicted=cf_predictions)
content_personalization = recmetrics.personalization(predicted=content_predictions)
ngcf_personalization = recmetrics.personalization(predicted=ngcf_predictions)
ngcf_embedding_personalization = recmetrics.personalization(predicted=ngcf_embedding_predictions)
print("Personalization")
print("Random :",random_personalization)
print("Popularity :", pop_personalization)
print("Collaborative Filtering :", cf_personalization)
print("Content Based Recommendation :", content_personalization)
print("NGCF :", ngcf_personalization)
print("NGCF Embedding :", ngcf_embedding_personalization)

#All Metrics 

In [ ]:
dic = {}
dic["Model"] = ['Random Recommender', 'Popularity Recommender', 'Collaborative Filtering', 'Content Based Recommender', 'NGCF (Random Embeddings)', 'NGCF (Custom Embeddings)']
dic["MAR@10"] = [random_mark[-1], pop_mark[-1], cf_mark[-1], content_mark[-1], ngcf_mark[-1], ngcf_embedding_mark[-1]]
dic["MAP@10"] = [random_mapk[-1], pop_mapk[-1], cf_mapk[-1], content_mapk[-1], ngcf_mapk[-1], ngcf_embedding_mapk[-1]]
dic["MRR"] = [random_mrr, pop_mrr, cf_mrr, content_mrr, ngcf_mrr, ngcf_embedding_mrr]
if val == 0:
    dic["Coverage (%)"] = [random_coverage, pop_coverage, cf_coverage, content_coverage, ngcf_coverage, ngcf_embedding_coverage]
evaluation_df = pd.DataFrame.from_dict(dic, orient='columns')
evaluation_df